In [1]:
import openai
from openai import AzureOpenAI

ENDPOINT = "https://mango-bush-0a9e12903.5.azurestaticapps.net/api/v1"
API_KEY = "b7cf07cd-cb5d-4994-a95a-3d5dd5ae7c53"

API_VERSION = "2024-02-01"
MODEL_NAME = "gpt-4o"

client = AzureOpenAI(
    azure_endpoint=ENDPOINT,
    api_key=API_KEY,
    api_version=API_VERSION,
)

MESSAGES = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {
        "role": "assistant",
        "content": "The Los Angeles Dodgers won the World Series in 2020.",
    },
    {"role": "user", "content": "Where was it played?"},
]

completion = client.chat.completions.create(
    model=MODEL_NAME,
    messages=MESSAGES,
)

print(completion.model_dump_json(indent=2))

{
  "id": "chatcmpl-ARQt55c8SwfwbJsOmrQoyyQy5hUT1",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The 2020 World Series was played at Globe Life Field in Arlington, Texas. This marked the first time in history that the entire World Series was played at a neutral site, due to the COVID-19 pandemic.",
        "refusal": null,
        "role": "assistant",
        "audio": null,
        "function_call": null,
        "tool_calls": null
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "created": 1731101575,
  "model": "gpt-4o-2024-05-13",


In [2]:
import pickle

with open("models_and_preprocessor/learning_pace.pkl", "rb") as file:
    model = pickle.load(file)
print(type(model))  # Should show <class 'sklearn.linear_model._logistic.LogisticRegression'>


<class 'sklearn.linear_model._logistic.LogisticRegression'>


/opt/anaconda3/envs/microsoft_hackthon/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.2.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [6]:
import pickle
import pandas as pd
import openai
from openai import AzureOpenAI

with open("models_and_preprocessor/learning_pace.pkl", "rb") as file:
    model = pickle.load(file)

def preprocess_student_data(student_data):
    encoded_data = {
        'Age': student_data['Age'],
        'Gender': 0 if student_data['Gender'] == 'Male' else 1,  # Example: Male=0, Female=1
        'ParentalEducation': {
            'HighSchool': 0, 'SomeCollege': 1, 'AssociateDegree': 2, 'Bachelor': 3, 'Master': 4
        }.get(student_data['ParentalEducation'], 0),  # Default to 0 if not found
        'Absences': student_data['Absences'],
        'Tutoring': 1 if student_data['Tutoring'] == 'Yes' else 0,
        'ParentalSupport': 1 if student_data['ParentalSupport'] == 'Yes' else 0,
        'Extracurricular': 1 if student_data['Extracurricular'] == 'Yes' else 0,
        'GPA': student_data['GPA']
    }
    return encoded_data

def predict_learning_pace(model, student_data):
    preprocessed_data = preprocess_student_data(student_data)
    student_df = pd.DataFrame([preprocessed_data])
    prediction = model.predict(student_df)[0]  # 0 for slow, 1 for fast
    print("fast learner" if prediction == 1 else "slow learner")
    return "fast learner" if prediction == 1 else "slow learner"

def generate_study_plan(student_input):
    learning_pace = predict_learning_pace(model, student_input)

    MESSAGES = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": f"I need help creating a study plan. Based on my habits, my predicted learning pace is {learning_pace}. I want to improve my study efficiency."},
        {
            "role": "assistant",
            "content": "Sure, I'd be happy to help! What subjects or topics do you need to include in your study plan?",
        },
        {"role": "user", "content": "The plan should cover mathematics over the next month."},
        {
            "role": "assistant",
            "content": "Do you have any specific goals or exams for these subjects during this period?",
        },
        {"role": "user", "content": "Yes, I'm preparing for midterms and need to cover calculus in mathematics."}
    ]

    client = AzureOpenAI(
        azure_endpoint="https://mango-bush-0a9e12903.5.azurestaticapps.net/api/v1",
        api_key="b7cf07cd-cb5d-4994-a95a-3d5dd5ae7c53",
        api_version="2024-02-01",
    )

    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=MESSAGES
    )

    response_content = completion.choices[0].message.content
    return response_content

# ex. student input 
student_input = {
    'Age': 17,
    'Gender': 'Male',
    'ParentalEducation': 'HighSchool',
    'Absences': 3,
    'Tutoring': 'Yes',
    'ParentalSupport': 'Yes',
    'Extracurricular': 'No',
    'GPA': 3.5
}

study_plan = generate_study_plan(student_input)
print(study_plan)


/opt/anaconda3/envs/microsoft_hackthon/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.2.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


fast learner


RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 8 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}}